In [6]:
import pandas as pd
import numpy as np

Пайплайн принимает в себя методы выбора, обработки данных, модель обучения и последовательно выполняет каждое действие. При этом он перенимает все методы модели.

В узком смысле пайплайн – это модуль sklearn.pipeline, который позволяет автоматизировать предварительные преобразования данных перед обучением модели.

In [2]:
# Импорт пайплайна
from sklearn.pipeline import Pipeline, make_pipeline

In [56]:
# Импорт классов для дополнительных преобразований
from sklearn.base import BaseEstimator, TransformerMixin

### Пример 1. Pipeline для векторного представления текста с TFIDF

In [57]:
df = pd.read_csv("support_data.csv").drop('Unnamed: 0', 1)
df.head(3)

,Title,Resolution,class,category
0,Need Ethernet access,Please connect the LAN cable to the red port o...,41,1
1,Create new account and share the username and ...,"Go to URL : accountsetup.com, Enter personal d...",35,0
2,FW: Myspace Login Issue-- Ania,"Go to URL : password.reset.com, Enter you logi...",36,0


Тут ради демонстрации Pipeline и tfidf возьмем признак Title и обработаем его

In [58]:
# импортируем tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['category'], random_state=0)

In [59]:
# Соберем простой пайплайн. Для этого напишем класс, который будет выбирать признак
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return X[self.column]

from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([('title_selector', FeatureSelector(column='Title')), # выбираем данные признака Title
                     ('tfidf', TfidfVectorizer()), # трансформируем их в вектора
                     ('classificator', LogisticRegression())], verbose = True)

In [60]:
# обучим пайплайн
pipeline.fit(X_train, y_train)

[Pipeline] .... (step 1 of 3) Processing title_selector, total=   0.0s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ..... (step 3 of 3) Processing classificator, total=   0.0s


Pipeline(steps=[('title_selector', FeatureSelector(column='Title')),
                ('tfidf', TfidfVectorizer()),
                ('classificator', LogisticRegression())],
         verbose=True)

In [62]:
pipeline.predict_proba(X_test)[:10,:]

array([[0.91081012, 0.08918988],
       [0.92431379, 0.07568621],
       [0.76492273, 0.23507727],
       [0.83646514, 0.16353486],
       [0.76874921, 0.23125079],
       [0.91265265, 0.08734735],
       [0.9098258 , 0.0901742 ],
       [0.91546489, 0.08453511],
       [0.71695714, 0.28304286],
       [0.90211255, 0.09788745]])

In [64]:
preds = pipeline.predict_proba(X_test)[:,1]
precision, recall, thresholds = precision_recall_curve(y_test, preds)

In [65]:
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.214648, F-Score=0.960, Precision=0.923, Recall=1.000


Как это интерпретировать?

Допустим, что у нас в тесте 1000 заявок. Из них класс 1 - 200

Мы обучили модель, сделали прогноз, посчитали порог и выше этого порога у нас 100 заявок.

При этом оказывается, что среди этих 100 заявок - 80 действительно класс 1.

TP = 80, FP = 20, TN = 780, FN = 120

Precision = TP/(TP+FP) = 80/(80+20) = 0.8

Recall = TP/(TP+FN) = 80/(80+120) = 0.4

### Пример 2. Поочередное применение пайплайнов до модели

In [17]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [39]:
# сразу разделим нашим данные на трейн и тест
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio',1), df['cardio'], test_size = 0.3, random_state = 42)

Напишем несколько классов для выбора и обработки данных, но сначала нам нужно понять, что делать

К полям:
- gender, cholesterol применим OHE-кодирование(это dummies)
- age, height, weight, ap_hi, ap_lo - StandardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [18]:
# Трансформатор для выбора одного столбца из фрейма данных для выполнения дополнительных преобразований 
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
# Выбор значений числового признака
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
# Dummies кодирование
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
# Отсортируем наши признаки для разных пайплайнов
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']
# Пустые списки для трансформированных данных
continuos_transformers = []
cat_transformers = []
base_transformers = []

Теперь запустим нашу обработку, в рамках которой и соберем пайплайны

In [21]:
for cont_col in continuos_cols:
    # создаем объект класса Pipeline
    transfomer =  Pipeline([
        # вызываем класс выбора признака для значений признака cont_col
                ('selector', NumberSelector(key=cont_col)),
        # применяем стандартизацию для выбранного признака
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    


    
for cat_col in cat_cols:
    # создаем объект класса Pipeline
    cat_transformer = Pipeline([
        # вызываем класс выбора признаков из cat_col
                ('selector', ColumnSelector(key=cat_col)),
        # преобразуем их в dummies переменные
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
    
for base_col in base_cols:
    # создаем объекст класса Pipeline
    base_transformer = Pipeline([
        ('selector', NumberSelector(key = base_col)),
    ])
    base_transformers.append((base_col, base_transformer))

In [32]:
# На выходе имеем список кортежей ('признак', Pipeline обработки)
continuos_transformers

[('age',
  Pipeline(steps=[('selector', NumberSelector(key='age')),
                  ('standard', StandardScaler())])),
 ('height',
  Pipeline(steps=[('selector', NumberSelector(key='height')),
                  ('standard', StandardScaler())])),
 ('weight',
  Pipeline(steps=[('selector', NumberSelector(key='weight')),
                  ('standard', StandardScaler())])),
 ('ap_hi',
  Pipeline(steps=[('selector', NumberSelector(key='ap_hi')),
                  ('standard', StandardScaler())])),
 ('ap_lo',
  Pipeline(steps=[('selector', NumberSelector(key='ap_lo')),
                  ('standard', StandardScaler())]))]

Теперь необходимо объединить наши трансформеры в единое целое

В этом нам поможет FeatureUnion

In [25]:
from sklearn.pipeline import FeatureUnion

In [26]:
# объединим все трансформеры
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feats

FeatureUnion(transformer_list=[('age',
                                Pipeline(steps=[('selector',
                                                 NumberSelector(key='age')),
                                                ('standard',
                                                 StandardScaler())])),
                               ('height',
                                Pipeline(steps=[('selector',
                                                 NumberSelector(key='height')),
                                                ('standard',
                                                 StandardScaler())])),
                               ('weight',
                                Pipeline(steps=[('selector',
                                                 NumberSelector(key='weight')),
                                                ('standard',
                                                 StandardScaler())])),
                               ('ap_hi',
                    

И запихнем все это в единый пайплайн
Тут в пайплайн в виде метода уже передается созданный список FeatureUnion, состоящий из нескольких Pipeline's с уже определенными нами методами для каждого признака

In [29]:
feature_processing = Pipeline([('feats', feats)])
feature_processing

Pipeline(steps=[('feats',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                           

Натренируем общий пайплайн( тоесть проведем обработку, вызвав метод fit_transform для X_train)

In [36]:
feature_processing.fit_transform(X_train)

array([[ 0.38817489, -0.5308834 , -0.70816849, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.30846829,  0.32314728, -0.63861364, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.34652706, -0.5308834 , -0.56905879, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.67245587,  0.56715605, -0.01261999, ...,  0.        ,
         0.        ,  1.        ],
       [-0.49851342,  0.32314728, -0.29083939, ...,  0.        ,
         0.        ,  0.        ],
       [-1.75890645,  1.54319112, -0.70816849, ...,  0.        ,
         0.        ,  1.        ]])

Теперь добавим классификатор в наш пайплайн

In [38]:
classifier = Pipeline([('features', feats),
                      ('classifier', LogisticRegression())])

### Запустим кросс-валидацию

In [40]:
cv_scores = cross_val_score(classifier, X_train, y_train, cv = 16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score равен {} с std {}'.format(cv_score, cv_score_std))

CV score равен 0.7800680715650671 с std 0.008891911952969385


Обучим модель на всем тренировочном сете

In [41]:
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:,1]

Посчитаем precision,recall и trashold

In [43]:
from sklearn.metrics import precision_recall_curve
# найдем точность, полноту и порог
precision, recall, threshold = precision_recall_curve(y_test.values, y_score)
# посчитаем f_score без дизбаланса точности/полноты
b = 1
fscore = (1+b**2)*(precision * recall)/(b**2*precision + recall)
# возьмем индексы самого большого значения f_score. Они же буду отвечать оптимальным значения в precision, recall и trashold
ix = np.argmax(fscore)
print('Лучший threshold {}  f_score  {}  precision  {}  recall  {}'.format(threshold[ix], fscore[ix], precision[ix], recall[ix]))

Лучший trashold 0.4101026847780329  f_score  0.7343882213572517  precision  0.6626708406505306  recall  0.8235126672359806
